In [1]:
# reference
# [RDKitで配座を発生させてエネルギーを計算してみる – Is life worth living?](https://iwatobipen.wordpress.com/2012/10/21/rdkit%E3%81%A7%E9%85%8D%E5%BA%A7%E3%82%92%E7%99%BA%E7%94%9F%E3%81%95%E3%81%9B%E3%81%A6%E3%82%A8%E3%83%8D%E3%83%AB%E3%82%AE%E3%83%BC%E3%82%92%E8%A8%88%E7%AE%97%E3%81%97%E3%81%A6%E3%81%BF%E3%82%8B/)

import sys
from rdkit import Chem
from rdkit.Chem import AllChem

RDKit WARNING: [08:10:59] Enabling RDKit 2019.09.2 jupyter extensions


In [2]:
# sdf = Chem.SDMolSupplier(sys.argv[1])
sdf = Chem.SDMolSupplier("./sdf/ArBr.sdf")
mols = [Chem.AddHs(m) for m in sdf]
out = Chem.SDWriter("./data/genconf.sdf")

In [3]:
mols

In [4]:
for mol in mols:
    ids = AllChem.EmbedMultipleConfs(mol, numConfs = 10, clearConfs = True, pruneRmsThresh = 0.5)
    for cid in ids:
        AllChem.UFFOptimizeMolecule(mol, maxIters = 1000, confId = cid)
        ff = AllChem.UFFGetMoleculeForceField(mol, confId = cid)
        ff.Minimize()
        energy=ff.CalcEnergy()
        mol.SetProp("UFF_Energy", "%s"%energy)
        mol.SetProp("confId", "%s" % cid)
        out.write(mol, confId=cid)
out.close()